In [2]:
import time
import requests
import lxml.html
import re

In [3]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1) # 1초간 대기
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력
        break  # for 루프 종료.

In [4]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [5]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
            for p in root.cssselect('#tabs_1 .hanbit_edit_view p') # 상세 페이지의 목차를 가져온다.
            if normalize_spaces(p.text_content()) != ''] # 공백문자를 지우기 위한 과정. (공백이 두 칸 이상이면 한 칸으로 줄이기 위함.)
    }
    return ebook

In [6]:
def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', ' ', s).strip()

In [7]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B7269609529', 'title': '나는 꼭 필요한 것만 남기기로 했다', 'price': '12,150', 'content': ['어느 순간', '주변 가득한 물건들이 내 공간을 침범했다', '최근 tvN 예능프로그램 <신박한 정리>가 연일 화제다. ‘이건 추억이 담긴 물건이라’ ‘아이들이 많다 보니’ 꽉 들어찬 공간에는 저마다 사연이 있지만, 말끔히 정리된 집 안을 보고 의뢰인들은 눈물까지 글썽인다.', '그렇지 않아도 코로나 사태 장기화로 집에 있는 시간이 많아지면서 평소에는 크게 신경 쓰지 않던 것들이 눈에 들어온다. 옷이 잔뜩 들어 있어 미어터질 지경인 옷장이나, 더는 책을 꽂을 데도 없고 내려앉기 일보직전인 책장, 잡동사니가 굴러다니는 서랍, 며칠째 뜯지도 않은 택배 상자….', '이대로는 안 되겠다 싶어 정리를 결심하지만, 어디서부터 어떻게 손을 대야 할지 난감한 경우가 대다수. 그렇다고 우리 모두가 유명 프로그램이나 전문가의 도움을 받을 수 있는 것도 아니지 않은가.', '이 책 《나는 꼭 필요한 것만 남기기로 했다》는 이제 막 ‘비우고 정리하기’를 결심한 사람들을 위한 책이다. 별 생각 없이 사놓고 몇 번 쓰지도 않은 채 던져두거나, 어딘가에 처박아놓고 까맣게 잊어버린 뒤 똑같은 물건을 산 경험이 있다면, 정리가 필요한 시점임에 틀림없다. 집 안 가득한 물건 탓에 서서 잠을 잘 지경이 되어서야 정리를 시작할 것인가.', '이 책은 미니멀 라이프를 꿈꾸는 사람들을 위한 21일간의 맞춤형 프로그램이다. 21일이라는 시간 동안 저자와 함께 과제를 수행하고, 행동으로 실천한 뒤 결과를 평가해보면서 자연스레 비우고 정리해보자. 그러면 어느새 자신에게 꼭 필요한 것만 남게 된다.', '공간의 여유가 마음의 여유를 만든다', '이 책의 저자도 처음부터 미니멀리스트는 아니었다. 그녀 역시 물건을 사고 소유하는 것이 만병통치약이나 다름없었다고 